#### Transforming the problem into the barrier option type. 

### Description:
* Consider a new payoff function $g(S_{1:M}) = \sum_{i=1}^{\tau} f(S_{i}) + 100$. Then $g \geq 0$.
* We divide the state sace $\mathbb{R}^{m}$ into two parts, one where the new payoff is 0 (ie when a particle does not escape the piy by time 5) and the other where the payoff is positive. 
* When a particle does not escape by time 5, we resample it from among the ones that have escaped. 

In [1]:
# Calling libraries:
%matplotlib inline  
from __future__ import division
import numpy as np
import time
from scipy.stats import norm, uniform
from pylab import plot, show, legend
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (10.0, 3.0) #for larger plots
int = np.vectorize(int)

# Initialization:
m, k, M = 24, 30, 10**5
gamma_L, gamma_G = 100, 200
S_0, mu, sigma = np.log(100), 0, 0.08

# Payoff function:
def f(x):
    s = np.exp(x)
    return (2*(s-110)+20)*(s>110) + (2*(80-s)+20)*(s<90) - 20*(s<=110)*(s>=90)  
def f_positive(x):
    return f(x)*(f(x)>0)
def f_negative(x):
    return -f(x)*(f(x)<0)
def payoff_function(x):
    return 100 + f(x)
np.vectorize(f)
np.vectorize(f_positive)
np.vectorize(f_negative)
np.vectorize(payoff_function)

In [2]:
# Systematic resampling function: 
def systematic_resampling(population, weights, index_resampled):
    weights /= np.sum(weights)
    population_length = len(population)
    u = np.random.random() / population_length
    p_index=0; sum_weights = 0
    for k in xrange(population_length):
        sum_weights += weights[k]
        while (u < sum_weights ):
            index_resampled[p_index] = k
            p_index+=1
            u = u + 1.0/population_length

In [3]:
# Naive Monte Carlo for TARN price:
def MC_TARN(m,k,S_0,sigma,mu,M,gamma_G,gamma_L):
    S, G, L = [S_0]*M, np.zeros(M), np.zeros(M)
    price, survive = np.zeros(M), np.ones(M)
    mean, sd = (mu-0.5*sigma**2)*k/360, sigma*np.sqrt(k/360)
    for n in range(m):
        S += mean + sd*norm.rvs(0,1,M)
        price += f(S)*survive
        G += f_positive(S)*survive
        L += f_negative(S)*survive
        survive *= (G<gamma_G)*(L<gamma_L)     # these are the ones which are still 'alive' after time n
    return np.mean(price) + 100

In [12]:
# SMC: 

def SMC_TARN(m,k,S_0,sigma,mu,M,gamma_G,gamma_L):
    
    S, G, L = [S_0]*M, np.zeros(M), np.zeros(M)
    weights, normalizing_constant = [1/M]*M, 1.0
    survive, payoff_S, index_resampled = np.ones(M), np.zeros(M), int(np.ones(M))
    mean, sd = (mu-0.5*sigma**2)*k/360, sigma*np.sqrt(k/360)
    
    for n in range(5):
        S += mean + sd*norm.rvs(0,1,M)
        payoff_S += f(S)*survive
        G += f_positive(S)*survive
        L += f_negative(S)*survive
        survive *= (G<gamma_G)*(L<gamma_L) 
    payoff_S += 100 
    weights *= payoff_S*(L<gamma_L)
    n_escaped = np.sum(L<gamma_L)
    normalizing_constant *= np.sum(weights)
    weights /= np.sum(weights)
    systematic_resampling(S,weights,index_resampled) 
    S, payoff_S, G, L = S[index_resampled], payoff_S[index_resampled], G[index_resampled], L[index_resampled]
    survive, weights = survive[index_resampled], 1/M
    
    weights /= payoff_S
    for n in np.add(5,range(m-5)):
        S += mean + sd*norm.rvs(0,1,M)
        payoff_S += f(S)*survive
        G += f_positive(S)*survive
        L += f_negative(S)*survive
        survive *= (G<gamma_G)*(L<gamma_L) 
    weights *= payoff_S
    normalizing_constant *= np.sum(weights)
    weights /= np.sum(weights)
    
    return normalizing_constant, n_escaped

In [5]:
# Sanity check:

MC_TARN(m,k,S_0,sigma,mu,M,gamma_G,gamma_L), SMC_TARN(m,k,S_0,sigma,mu,10**4,gamma_G,gamma_L)

(7.0961396061894106, 7.1783713279869943)

### Variance comparison:

In [6]:
def compare_methods(rep,M):
    MC_TARN_estimate, SMC_TARN_estimate = np.zeros(rep), np.zeros(rep)
    start_time = time.clock()
    for r in range(rep):
        MC_TARN_estimate[r] = MC_TARN(m,k,S_0,sigma,mu,M,gamma_G,gamma_L)
    MC_time = time.clock() - start_time 
    start_time = time.clock()
    for r in range(rep):
        SMC_TARN_estimate[r] = SMC_TARN(m,k,S_0,sigma,mu,M,gamma_G,gamma_L)[0]
    SMC_time = time.clock() - start_time 
    print "sigma =", sigma, ", no. of particles =", M
    print "Average time for one run of MC, SMC =", round(MC_time/rep,2), ",", round(SMC_time/rep,2), "sec."
    print "Mean of MC, SMC =", round(np.mean(MC_TARN_estimate),2), ",", round(np.mean(SMC_TARN_estimate),2)
    print "Relative standard deviation of MC to SMC =", round(np.std(MC_TARN_estimate)/np.std(SMC_TARN_estimate),2)
    print "Total time for", rep, "repetitions =", round((MC_time+SMC_time)/60,2), "minutes"

In [7]:
sigma = 0.08
compare_methods(rep=100,M=10**4)

sigma = 0.08 , no. of particles = 10000
Average time for one run of MC, SMC = 0.05 , 0.07 sec.
Mean of MC, SMC = 7.14 , 7.11
Relative standard deviation of MC to SMC = 1.02
Total time for 100 repetitions = 0.2 minutes


In [8]:
sigma = 0.02
compare_methods(rep=100,M=10**5)

-c:19: RuntimeWarning: invalid value encountered in true_divide
-c:24: RuntimeWarning: divide by zero encountered in true_divide
-c:31: RuntimeWarning: invalid value encountered in multiply


sigma = 0.02 , no. of particles = 100000
Average time for one run of MC, SMC = 0.64 , 0.58 sec.
Mean of MC, SMC = 0.0 , nan
Relative standard deviation of MC to SMC = nan
Total time for 100 repetitions = 2.04 minutes


In [9]:
sigma = 0.03
compare_methods(rep=100,M=10**5)

sigma = 0.03 , no. of particles = 100000
Average time for one run of MC, SMC = 0.56 , 0.65 sec.
Mean of MC, SMC = 0.0 , nan
Relative standard deviation of MC to SMC = nan
Total time for 100 repetitions = 2.01 minutes


In [10]:
sigma = 0.04
compare_methods(rep=100,M=10**5)

sigma = 0.04 , no. of particles = 100000
Average time for one run of MC, SMC = 0.51 , 0.57 sec.
Mean of MC, SMC = 0.01 , 0.01
Relative standard deviation of MC to SMC = 1.22
Total time for 100 repetitions = 1.81 minutes


In [11]:
sigma = 0.05
compare_methods(rep=100,M=10**5)

sigma = 0.05 , no. of particles = 100000
Average time for one run of MC, SMC = 0.51 , 0.58 sec.
Mean of MC, SMC = 0.23 , 0.23
Relative standard deviation of MC to SMC = 1.05
Total time for 100 repetitions = 1.81 minutes


In [15]:
sigma = 0.04
SMC_TARN(m,k,S_0,sigma,mu,10**5,gamma_G,gamma_L)

(0.0127394227789838, 13)

In [20]:
sigma = 0.05
SMC_TARN(m,k,S_0,sigma,mu,10**5,gamma_G,gamma_L)

(0.23834415221107519, 241)